In [2]:
import pickle
from sklearn import cross_validation
import pandas as pd

In [17]:
df_art = pickle.load(open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/df.p', 'rb'))

In [18]:
df_journals = pickle.load(open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/pickled_objects/dfJournals_trans_categories.p', 'rb'))

In [19]:
print df_art.shape
print df_journals.shape
print df_art.columns.values
print df_journals.columns.values

df_art = df_art.drop('pmid', axis=1)
print df_art.columns.values
#for the journals df, change the label of the 'sentiment' column to 'label'
df_journals.columns = ['label', 'title', 'abstract', 'qualifier_terms', 'descriptor_terms']
#reorder
df_journals = df_journals[df_art.columns.values]
print df_journals.columns.values

(354, 6)
(97049, 5)
['pmid' 'title' 'abstract' 'qualifier_terms' 'descriptor_terms' 'label']
['sentiment' 'title' 'abstract' 'qualifier_terms' 'descriptor_terms']
['title' 'abstract' 'qualifier_terms' 'descriptor_terms' 'label']
['title' 'abstract' 'qualifier_terms' 'descriptor_terms' 'label']


In [20]:
print df_art['label'].unique()
print df_journals['label'].unique()

['T0' 'T1/T2' 'T3/T4' 'TX']
['T0' 'T1/T2' 'T3/T4']


In [21]:
#add boolean labels
df_art['boolean_label'] = 1
for row in df_art.iterrows():
    if row[1].loc['label'] != 'T0':
        df_art.set_value(row[0], 'boolean_label', 0)
        
df_journals['boolean_label'] = 1
for row in df_journals.iterrows():
    if row[1].loc['label'] != 'T0':
        df_journals.set_value(row[0], 'boolean_label', 0)

In [22]:
df_art[0:5]

,title,abstract,qualifier_terms,descriptor_terms,label,boolean_label
0,Gene expression profiles in peripheral blood m...,Occupational exposure to nickel (Ni) is associ...,genetics metabolism epidemiology drug effects ...,Adult Asian Continental Ancestry Group Biomark...,T0,1
1,SOX2 is a cancer-specific regulator of tumour ...,Although the principles that balance stem cell...,genetics metabolism pathology metabolism patho...,"Animals Carcinoma, Squamous Cell Cell Line, Tu...",T0,1
2,Imaging and cerebrospinal fluid biomarkers in ...,The pathophysiological process of Alzheimer's ...,cerebrospinal fluid genetics radionuclide imag...,"Aged Aged, 80 and over Alzheimer Disease Amylo...",T0,1
3,Preliminary evidence of cognitive and brain ab...,To ascertain whether pediatric obesity without...,pathology physiopathology physiopathology,Adolescent Attention Brain Cognition Executive...,T0,1
4,Obese Adolescents with Type 2 Diabetes Mellitu...,The rates of type 2 diabetes (T2DM) continue t...,,,T0,1


In [23]:
df_journals[150:155]

,title,abstract,qualifier_terms,descriptor_terms,label,boolean_label
150,Identifying Direct Downstream Targets: WT1 ChI...,Identifying targets of transcriptional regulat...,,,T0,1
151,Measuring Equilibrium Binding Constants for th...,Equilibrium binding of WT1 to specific sites i...,,,T0,1
152,In Vitro Transcription to Study WT1 Function.,In vitro transcription methods using mammalian...,,,T0,1
153,Fluorescence-Activated Cell Sorting (FACS) Pro...,Zebrafish is becoming a very important model f...,,,T0,1
154,Immunofluorescence Staining of Wt1 on Sections...,Immunohistochemistry is one of the most powerf...,,,T0,1


### Combine 2 dfs together to make a training set, and 2 separate validation sets

In [46]:
#get journals training/test set (size 400)
skf = cross_validation.StratifiedShuffleSplit(df_journals['label'].values, n_iter=1, test_size=404)
for train_index, test_index in skf:
    train_journals = df_journals.iloc[test_index]
    df_journals_left = df_journals.iloc[train_index]
    print len(train_journals)

train_journals['dataset'] = 'journal'
    
#get journals validation set (size 157) from what's left of dataset
skf = cross_validation.StratifiedShuffleSplit(df_journals_left['label'].values, n_iter=1, test_size=152)
for train_index, test_index in skf:
    val_journals = df_journals_left.iloc[test_index]
    print len(val_journals)

404
152


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
train_journals[0:5]

,title,abstract,qualifier_terms,descriptor_terms,label,boolean_label,dataset
84953,The Impact of Chronic Liraglutide Therapy on G...,"In patients with type 2 diabetes (T2DM), impai...",metabolism blood blood drug therapy blood ther...,Aged Blood Glucose C-Peptide Diabetes Mellitus...,T3/T4,0,journal
64987,Right-siting chronic kidney disease care-a sur...,Chronic kidney disease (CKD) is a major public...,utilization statistics & numerical data diagno...,Ambulatory Care Facilities Female General Prac...,T3/T4,0,journal
27924,A genome-wide RNA interference screen identifi...,Tumor necrosis factor alpha (TNF-α) is a poten...,genetics immunology methods genetics immunolog...,"Caspases, Initiator Genomics HEK293 Cells HeLa...",T0,1,journal
18418,A C-terminal acidic domain regulates degradati...,Bob1 (Obf-1 or OCA-B) is a 34-kDa transcriptio...,physiology metabolism genetics metabolism meta...,Amino Acid Sequence Animals B-Lymphocytes Biom...,T0,1,journal
65411,Mixed methods research in music therapy research.,Music therapists have an ethical and professio...,organization & administration methods methods,Attitude to Health Benchmarking Biomedical Res...,T3/T4,0,journal


In [47]:
#get article training/test set (size 200) and validation set (size 157)
skf = cross_validation.StratifiedShuffleSplit(df_art['label'].values, n_iter=1, test_size=152)
for train_index, test_index in skf:
    train_art = df_art.iloc[train_index]
    val_art = df_art.iloc[test_index]
    print len(train_art)
    print len(val_art)

train_art['dataset'] = 'article'

202
152


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [28]:
train_art[0:5]

,title,abstract,qualifier_terms,descriptor_terms,label,boolean_label,dataset
54,Pulmonary hypertension and other potentially f...,Systemic juvenile idiopathic arthritis (JIA) i...,complications diagnosis mortality diagnosis et...,"Adolescent Arthritis, Juvenile Child Child, Pr...",T0,1,article
270,Are increased weight and appetite useful indic...,"During childhood and adolescence, physiologica...",physiology diagnosis physiopathology physiology,"Adolescent Appetite Child Child, Preschool Dep...",T0,1,article
191,How physician electronic health record screen ...,Use of electronic health records (EHRs) in pri...,,,T3/T4,0,article
213,Early identification and treatment of Alzheime...,Alzheimer's disease (AD) is a progressive neur...,diagnosis economics psychology therapy economi...,Alzheimer Disease Caregivers Cost Savings Cost...,T3/T4,0,article
320,Caregiver person-centeredness and behavioral s...,Mealtime behavioral symptoms are distressing a...,,,T3/T4,0,article


In [36]:
#test if properly stratified
frames = [train_art, train_journals]
df = pd.concat(frames)
print df.shape

(606, 7)


In [32]:
df[200:205]

,title,abstract,qualifier_terms,descriptor_terms,label,boolean_label,dataset
301,Importance of low preoperative platelet count ...,Low platelet count is a marker of portal hyper...,blood surgery blood surgery,"Adult Aged Carcinoma, Hepatocellular Cohort St...",T3/T4,0,article
187,Community consultation and public disclosure: ...,Emergency medicine research conducted under th...,ethics legislation & jurisprudence organizatio...,Community-Based Participatory Research Disclos...,T3/T4,0,article
84953,The Impact of Chronic Liraglutide Therapy on G...,"In patients with type 2 diabetes (T2DM), impai...",metabolism blood blood drug therapy blood ther...,Aged Blood Glucose C-Peptide Diabetes Mellitus...,T3/T4,0,journal
64987,Right-siting chronic kidney disease care-a sur...,Chronic kidney disease (CKD) is a major public...,utilization statistics & numerical data diagno...,Ambulatory Care Facilities Female General Prac...,T3/T4,0,journal
27924,A genome-wide RNA interference screen identifi...,Tumor necrosis factor alpha (TNF-α) is a poten...,genetics immunology methods genetics immunolog...,"Caspases, Initiator Genomics HEK293 Cells HeLa...",T0,1,journal


In [ ]:
#input to algorithm
#what algorithm? our multiclass random forest that mimics that of the original paper

In [51]:
pickle.dump(df, open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/mixed_journals_article/df.p', 'wb'))

In [52]:
pickle.dump(val_journals, open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/mixed_journals_article/val_journals.p', 'wb'))
pickle.dump(val_art, open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/mixed_journals_article/val_art.p', 'wb'))

Bootstrapping: include 2 copies of df_art in combined df:

In [50]:
#test if properly stratified
train_art_2 = train_art
train_art_3 = train_art
frames = [train_art, train_journals, train_art_2, train_art_3]
df = pd.concat(frames)
print df.shape

(1010, 7)
